In [80]:
import os
import re
import time
import codecs
import numpy as np
import pandas as pd
from zipfile import ZipFile
from sqlalchemy import create_engine

In [81]:
# today
today = time.strftime('%Y-%m-%d', time.localtime())

In [82]:
logfile = 'E:\\Log\\dns\\hkdc01dns.zip'
backup_path = 'E:\\Log\\backup\\'
sqldb = 'E:\\log\\windns-log.db'

In [83]:
disk_engine = create_engine('sqlite:///'+sqldb, encoding = 'UTF-8')
#disk_engine = create_engine('sqlite:///'+sqldb)
#store = pd.HDFStore('windns-log.h5')

In [84]:
names = ['Date', 'Threadid', 'Context', 'Intpacketid', 'Protocol', 'Direction', 'Remoteip', 'Xid', 'QueryResponse',
        'Opcode', 'Flag', 'Author', 'Trun', 'Recurd', 'Recura', 'Response', 'Questiontype', 'Domain']

In [85]:
dtype = {'Threadid':str, 'Context':str, 'Intpacketid':str, 'Protocol':str, 
         'Direction':str, 'Remoteip':str, 'Xid':str, 'QueryResponse':str,
        'Opcode':str, 'Flag':str, 'Author':str, 'Trun':str, 'Recurd':str, 
         'Recura':str, 'Response':str, 'Questiontype':str, 'Domain':str}

In [86]:
colspec = [(0, 20),(21, 25),(26, 34),(35, 51),(51, 54),(55, 58),(59, 75),(75, 79),(79, 80),
    (80, 81),(81, 82),(85, 89),(89, 90),(90, 91),(91, 92),(94, 103),(105, 112),(112, 200)]

In [87]:
start_time = time.time()
ret = []
with ZipFile(logfile, 'r') as zip:
   for filename in zip.namelist():
        if not os.path.isdir(filename):
            with zip.open(filename, 'rU') as f:
                #for line in f:
                for line in codecs.iterdecode(f, 'utf8'):
                    if '[' in line and ']' in line:
                        temp = []
                        for i in colspec:
                            #temp.append(re.sub(r'[\x03]|[\x8d]|[\xa0-\xaf]|[\xb0-\xbf]|[\xc0-\xcf]|[\xd0-\xdf]', 
                            #                   '',line[i[0]:i[1]].strip()))
                            temp.append(line[i[0]:i[1]])
                        ret.append(temp)
#df = pd.read_fwf(zip.open(filename, 'r'), colspecs=colspec, skiprows=29, skip_blank_lines=True)  
print ("file read in: " + str(time.time() - start_time) + " seconds")

file read in: 232.896999836 seconds


In [88]:
#ret[473000:473150]

In [89]:
start_time = time.time()
#df = pd.read_fwf('./hkdc01dns2.zip', colspecs=colspec, names=names, skiprows=29, skip_blank_lines=True, compression='zip')
df = pd.DataFrame(ret, columns=names)
print ("changed to dataframe in: " + str(time.time() - start_time) + " seconds")

changed to dataframe in: 13.9450001717 seconds


In [90]:
start_time = time.time()
df.dropna(axis=0, how='all', inplace=True) # somehow the skip_blank_lines is not working, so manually remove blank rows
df = df[df['Context'] != 'EVENT'] # filter out the EVENT row
#df['Domain'] = df['Domain'].str.replace('\x03', '?') # replace the unreadable char in the Domain column with '?'
#df['Domain'] = df['Domain'].str.replace('\(\d*\)', '.')
#df[names[1:]] = df[names[1:]].astype(str) # change all columns to string type
print ("data cleanup in: " + str(time.time() - start_time) + " seconds")

data cleanup in: 44.6449999809 seconds


In [91]:
#df.shape

In [92]:
#df['Domain'].str.replace('\\\\x', '?')[1457093:1457096]

In [93]:
#line = '(13)\xc2\xb5\xc2\xb1\xc2\xbe\xc3\xa2p\xc3\xb4\xc2\x8d(5)bdphk(3)com(0)'
#re.sub(u'\\xc2', '?', line)

In [94]:
#def clean_text(row):
# return the list of decoded cell in the Series instead 
#    return [r.decode('unicode_escape').encode('ascii', 'ignore') for r in row]

In [95]:
#df['Domain'] = df['Domain'].apply(clean_text)
#df['Response'] = df['Response'].apply(clean_text)

In [96]:
start_time = time.time()
df.to_sql('log', disk_engine, index=False, if_exists='append', chunksize=500)
#df[473000:473150].to_sql('log', disk_engine, index=False, if_exists='append', chunksize=500)
#df.to_hdf(store, key='log', format='table', mode='a', append=True, complevel=3, complib='zlib')
#df.to_csv('windns-log.csv')
print ("output data to sql in: " + str(time.time() - start_time) + " seconds")

output data to sql in: 218.404000044 seconds


In [97]:
filename = backup_path+today+".zip"
if os.path.isfile(filename) != True:
    os.rename(logfile, backup_path+today+".zip")
else:
    os.unlink(backup_path+today+".zip")

In [98]:
#start_time = time.time()
#df.to_sql('log', disk_engine, index=False, if_exists='append')
#print time.time() - start_time, "seconds"

In [99]:
#df['Domain'].value_counts()[0:50]

In [ ]:
if __name__ == "__main__":

    if len(sys.argv) != 2:
        sys.exit("number of input parameters is incorrect")

    path = sys.argv[1]
    
    rename(path, newpath)